In [1]:
import pandas as pd
import numpy as np
import nltk
import itertools
from nltk.corpus import sentiwordnet as swn
from normalization import normalize_accented_characters, html_parser, strip_html
from utils import display_evaluation_metrics, display_confusion_matrix, display_classification_report
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
import dynet as dy
from nltk.corpus import wordnet as wn
from collections import defaultdict

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


## Movie review dataset

In [2]:
def prepare_movie_dataset(train_start,train_end,test_start,test_end) : 

    dataset = pd.read_csv(r'datasets/movie_reviews.csv')
    print('dataset size : ',dataset.shape[0])

    train_data = dataset[train_start:train_end]
    test_data = dataset[test_start:test_end]
    
    print('Train_X : ',train_data.shape[0])
    print('Test_X  : ',test_data.shape[0])

    test_reviews = np.array(test_data['review'])
    test_sentiments = np.array(test_data['sentiment'])

    return train_data,test_reviews,test_sentiments

def prepare_labeled_data(train_start,train_end,test_start,test_end) : 
    
    labeled_data=open("datasets/labeledTrainData.tsv","r")
    data=labeled_data.readlines()
    data=[d.split("\t") for d in data]
    sa_data=pd.DataFrame(data,columns=['ind','sentiment','review'])
    sa_data=sa_data[['sentiment','review']]
    
    print('dataset size : ',sa_data.shape[0])

    train_data = sa_data[train_start:train_end]
    test_data = sa_data[test_start:test_end]
    
    print('Train_X : ',train_data.shape[0])
    print('Test_X  : ',test_data.shape[0])

    test_reviews = np.array(test_data['review'])
    test_sentiments = np.array(test_data['sentiment'])

    return train_data,test_reviews,test_sentiments

In [3]:
train_x,test_x,test_y=prepare_movie_dataset(1,1000,1000,2000)

dataset size :  50000
Train_X :  999
Test_X  :  1000


## Evaluation for unsupervised Lexicon sentiment tagging

#### compare against the sentence tagging (already provided in the dataset )

[add markdown #11 here]

In [4]:
### BORROWED FROM THE AR_SARKAR METRIC
def analyze_sentiment_sentiwordnet_lexicon(review,verbose=False):
    
    
    #review = normalize_accented_characters(review)
    #review = review.decode('utf-8')
    review = html_parser.unescape(review)
    review = strip_html(review)
    
    text_tokens = nltk.word_tokenize(review)
    tagged_text = nltk.pos_tag(text_tokens)
    pos_score = neg_score = token_count = obj_score = 0

    for word, tag in tagged_text:
        ss_set = None
        if 'NN' in tag and swn.senti_synsets(word, 'n'):
            ss_set = list(swn.senti_synsets(word, 'n'))
            if ss_set : 
                ss_set=ss_set[0]
        elif 'VB' in tag and swn.senti_synsets(word, 'v'):
            ss_set = list(swn.senti_synsets(word, 'v'))
            if ss_set : 
                ss_set=ss_set[0]
        elif 'JJ' in tag and swn.senti_synsets(word, 'a'):
            ss_set = list(swn.senti_synsets(word, 'a'))
            if ss_set : 
                ss_set=ss_set[0]
        elif 'RB' in tag and swn.senti_synsets(word, 'r'):
            ss_set = list(swn.senti_synsets(word, 'r'))
            if ss_set : 
                ss_set=ss_set[0]
        
        if ss_set:
            
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    
    
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score,
                                         norm_pos_score, norm_neg_score,
                                         norm_final_score]],
                                         columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Objectivity',
                                                                       'Positive', 'Negative', 'Overall']], 
                                                              labels=[[0,0,0,0,0],[0,1,2,3,4]]))
        print (sentiment_frame)   
    return final_sentiment
            
                                                               
def evaluate_lexicons(TRUE_LABELS,PREDICTED_LABELS,POS_CLASS,NEG_CLASS) : 

    print ('Performance metrics:')
    display_evaluation_metrics(true_labels=TRUE_LABELS,
                               predicted_labels=PREDICTED_LABELS,
                               positive_class=str(POS_CLASS))  
    print ('\nConfusion Matrix:'             )              
    display_confusion_matrix(true_labels=TRUE_LABELS,
                             predicted_labels=PREDICTED_LABELS,
                             classes=[str(POS_CLASS),str(NEG_CLASS)])
    print ('\nClassification report:' )                        
    display_classification_report(true_labels=TRUE_LABELS,
                                  predicted_labels=PREDICTED_LABELS,
                                  classes=[str(POS_CLASS),str(NEG_CLASS)])
    return

                               

## Basline lexicon evaluation

#### movie dataset 

In [36]:
train_x,test_x,test_y=prepare_movie_dataset(0,1000,1000,2000)
sentiwordnet_predictions = [analyze_sentiment_sentiwordnet_lexicon(review) for review in test_x]
evaluate_lexicons(test_y.tolist(),sentiwordnet_predictions,'positive','negative')

dataset size :  50000
Train_X :  1000
Test_X  :  1000


KeyboardInterrupt: 

#### labeled dataset 

In [ ]:
train_x,test_x,test_y=prepare_labeled_data(0,1000,1000,2000)
sentiwordnet_predictions = [analyze_sentiment_sentiwordnet_lexicon(review) for review in test_x]
binary_predicted=['1' if p=='positive' else '0' for p in sentiwordnet_predictions ]
evaluate_lexicons(test_y.tolist(),binary_predicted,'1','0')

### Simple network for learning (do afterwards)

In [5]:
## SIMPLE NETWORK WITH THE sigma(V*tanh(WX+B)) ## for the XOR problem
# create a parameter collection and add the parameters.
m = dy.ParameterCollection()
W = m.add_parameters((8,2))
V = m.add_parameters((1,8))
b = m.add_parameters((8))

dy.renew_cg() # new computation graph. not strictly needed here, but good practice.
b.value() ## bias values
x=dy.vecInput(2) ## 2 sized inputs 
output=dy.logistic(V*(dy.tanh(W*x)+b)) ## output node

y = dy.scalarInput(0) ## objective function
loss = dy.binary_log_loss(output,y) ## loss function
## trainer with the initialized parameters m 
trainer=dy.SimpleSGDTrainer(m)
x.set([1,0])
y.set(1)
loss_value = loss.value() # this performs a forward through the network.
print("the loss before step is:",loss_value)

loss.backward()  # compute the gradients
trainer.update()

loss_value = loss.value(recalculate=True) 
print("the loss after step is:",loss_value)
pc = dy.ParameterCollection()
NUM_LAYERS=2
INPUT_DIM=50
HIDDEN_DIM=10
builder = dy.LSTMBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, pc)
s0 = builder.initial_state()
x1 = dy.vecInput(INPUT_DIM)
s1=s0.add_input(x1)
y1 = s1.output()
s2=s1.add_input(x1) # we can add another input
y2=s2.output()

the loss before step is: 1.0331029891967773
the loss after step is: 0.7983773350715637


### IMPLEMENTING 2.1 SECTION OF GOOGLE PAPER FOR LEXICON EXPANSION [this is for the lexicon expansion ]

In [ ]:
ALL_WORDS=list(wn.words())
SCORES=defaultdict()
df=pd.DataFrame(ALL_WORDS,columns=['word'])

swn.all_
    

### Comparing the lexicons from stanford paper "Incuding Domain-Specific Sentiment Lexicons from Unalabeled Copora"

In [20]:
import socialsent_util
def load_lexicon(name, remove_neutral=True):
    lexicon = socialsent_util.load_json("./lexicons_socialsent/"+ name + '.json')
    return {w: p for w, p in lexicon.items() if p != 0} if remove_neutral else lexicon

def compare_lexicons(print_disagreements=False):
    lexicons = {
        "inquirer": load_lexicon("inquirer", False),
        "mpqa": load_lexicon("mpqa", False),
        "bingliu": load_lexicon("bingliu", False),
    }

    for l in lexicons:
        print( l, len(lexicons[l]), len([w for w in lexicons[l] if lexicons[l][w] != 0]))

    for l1, l2 in itertools.combinations(lexicons.keys(), 2):
        ps1, ps2 = lexicons[l1], lexicons[l2]
        common_words = set(ps1.keys()) & set(ps2.keys())
        print( l1, l2, "agreement: {:.2f}".format(
            100.0 * sum(1 if ps1[w] == ps2[w] else 0 for w in common_words) / len(common_words)))
        common_words = set([word for word in ps1.keys() if ps1[word] != 0]) & \
                       set([word for word in ps2.keys() if ps2[word] != 0])  
        print (l1, l2, "agreement ignoring neutral: {:.2f}".format(
            100.0 * sum(1 if ps1[w] * ps2[w] == 1 else 0 for w in common_words) / len(common_words)))
        
        if print_disagreements and l1 == 'opinion' and l2 == 'inquirer':
            for w in common_words:
                if lexicons[l1][w] != lexicons[l2][w]:
                    print (w, lexicons[l1][w], lexicons[l2][w])
      
    
## ALL THESE LEXICONS ARE 2-CLASS SENTIMENTS. 1 = POSITIVE; -1 = NEGATIVE
finance_lexicons=load_lexicon('finance')
bingliu_lexicons=load_lexicon('bingliu')
inquirer_lexicons=load_lexicon('inquirer')
mpqa_lexicons=load_lexicon('mpqa')
twitter_lexicons=load_lexicon('twitter')

In [21]:

ZEEYANG_LEXICONS='lexicons_zeeyang'
def read_zeeyang_lexicons(fname) : 
    
    polarities=defaultdict()
    for line in open(fname,'r') : 
        token=line.split(" ")[0]
        score=line.split(" ")[1]
        polarities[token]=score
        
    return polarities

## THESE LEXICONS HAVE CONTINOUS SCORES (BETWEEN -1 AND 1 )
senti140_lexicons=read_zeeyang_lexicons(ZEEYANG_LEXICONS+"/sentiment140.lex")
sentiwn_lexicons=read_zeeyang_lexicons(ZEEYANG_LEXICONS+"/sentiwordnet.lex")
sst_lexicons=read_zeeyang_lexicons(ZEEYANG_LEXICONS+"/stanford.tree.lexicon")



### Compares the different lexicon repositories through the mutual information between them (common words)

### The comparison is done through looking for words in two lexicon dictionaries L1 AND L2, and how many words are common in them which have the same scores.

In [7]:
## COMPARING THE BINARY LEXICONS 
compare_lexicons()



inquirer 8640 3457
mpqa 6886 6462
bingliu 6785 6785
inquirer mpqa agreement: 82.47
inquirer mpqa agreement ignoring neutral: 98.50
inquirer bingliu agreement: 84.39
inquirer bingliu agreement ignoring neutral: 98.74
mpqa bingliu agreement: 99.19
mpqa bingliu agreement ignoring neutral: 99.44


#### Lexicon Induction : the idea is to generate the lexicons provided the corpus. This method makes sure that the lexicon are sensitive to the context they are drawn from. They may prove useful if we would like to assess them in a simiar context. For instance, financial lexicons will reflect better sentiments than using general lexicons such as SentiWordNet. Three ways purposed for induction 


### POLARITY INDUCTION METHOD : This is used for re-scoring of the lexicons(tokens) by taking information from the word-embeddings (domain-specific), positive and the negative seed words.

In [8]:
import polarity_induction_methods

### THIS IS THE FUNCTION FOR INDUCING LEXICONS GIVEN THE SEEDS, EMBEDDINGS AND THE METHOD.
def run_method(positive_seeds, negative_seeds, embeddings, transform_embeddings=False, post_densify=False,
        method=polarity_induction_methods.densify, **kwargs):
    
    print("THE INTERNAL RUN_METHOD IS RUNNING...")
    
    if transform_embeddings:
        print ("Transforming embeddings...")
        embeddings = embedding_transformer.apply_embedding_transformation(embeddings, positive_seeds, negative_seeds, n_dim=50)
    
    print("AFTER EMBEDDING TRANSFORM ",embeddings)
    
    ## using densify method
    if post_densify:
        polarities = method(embeddings, positive_seeds, negative_seeds, **kwargs)
        top_pos = [word for word in 
                sorted(polarities, key = lambda w : -polarities[w])[:150]]
        top_neg = [word for word in 
                sorted(polarities, key = lambda w : polarities[w])[:150]]
        top_pos.extend(positive_seeds)
        top_neg.extend(negative_seeds)
        return polarity_induction_methods.densify(embeddings, top_pos, top_neg)
    
    
    positive_seeds = [s for s in positive_seeds if s in embeddings]
    negative_seeds = [s for s in negative_seeds if s in embeddings]
    
    
    return method(embeddings, positive_seeds, negative_seeds, **kwargs)


data =  /home/ubuntu/workspace/nlpclass-1187-g-Mad_Titans/sa/embeddings_socialsent/


Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## LEXICON INDUCTION ON STANDARD ENGLISH 

In [9]:
def calculate_new_lexicon_polarities(parent_lexicon,positive_seeds,negative_seeds,technique) : 
    
    ## CHOOSE SEEDS : 

    POSITIVE_SEEDS = ["good", "lovely", "excellent", "fortunate", "pleasant", "delightful", "perfect", "loved", "love", "happy"] 
    NEGATIVE_SEEDS = ["bad", "horrible", "poor",  "unfortunate", "unpleasant", "disgusting", "evil", "hated", "hate", "unhappy"]

    ## LOAD THE WORD-EMBEDDINGS : 

    eval_words = set(parent_lexicon.keys())

    EMBEDDING_TYPE = constants.GLOVE_EMBEDDINGS
    EMBEDDING = create_representation("GIGA", constants.GLOVE_EMBEDDINGS,eval_words.union(POSITIVE_SEEDS).union(NEGATIVE_SEEDS))

    embed_words = set(EMBEDDING.iw)
    eval_words = eval_words.intersection(EMBEDDING)
    eval_words = [word for word in eval_words  if not word in POSITIVE_SEEDS and not word in NEGATIVE_SEEDS]

    ## TRAIN THE BEST ALGORITHM : SENTPROP and get polarities re-scored
    
    polarities=defaultdict()
    if technique=='label_propagate_prob' : 
        
        polarities = run_method(POSITIVE_SEEDS, NEGATIVE_SEEDS, 
                    EMBEDDING.get_subembed(set(eval_words).union(NEGATIVE_SEEDS).union(POSITIVE_SEEDS)),
                    method=polarity_induction_methods.label_propagate_probabilistic,beta=0.99, nn=10)
    
    elif technique == 'pmi' : 
        
        EMBEDDING = create_representation("Explicit", constants.GLOVE_EMBEDDINGS)
        hist_counts = EMBEDDING.get_subembed(set(eval_words).union(positive_seeds).union(negative_seeds),restrict_context=False)
        
        
        print(dir(EMBEDDING))
        
        polarities = run_method(positive_seeds, negative_seeds,
                hist_counts,
                method=polarity_induction_methods.bootstrap,
                score_method=polarity_induction_methods.pmi)
    
    return polarities,eval_words

In [10]:
from representations.representation_factory import create_representation
import constants
from evaluate_methods import run_method
import polarity_induction_methods


## TRAINING THE LABEL-PROPAGATION FOR THE RE-SCORING OF POLARITIES FROM PRE-DETERMINED LEXICONS (MADE FROM WORD EMBEDDINGS)

INQUIRER = load_lexicon("inquirer", remove_neutral=False)

FINANCE_LEXICONS=load_lexicon('finance')
TWITTER_LEXICONS=load_lexicon('twitter')

THREE_WAY_LEXICON = kuperman = load_lexicon("kuperman", remove_neutral=False)

POSITIVE_FINANCE = ["successful", "excellent", "profit", "beneficial", "improving", "improved", "success", "gains", "positive"]
NEGATIVE_FINANCE = ["negligent", "loss", "volatile", "wrong", "losses", "damages", "bad", "litigation", "failure", "down", "negative"]


POSITIVE_SE = ["good", "lovely", "excellent", "fortunate", "pleasant", "delightful", "perfect", "loved", "love", "happy"] 
NEGATIVE_SE = ["bad", "horrible", "poor",  "unfortunate", "unpleasant", "disgusting", "evil", "hated", "hate", "unhappy"]



### Training financial and standard-english lexicons with glove embeddings using probabilistic label propagation method.

In [19]:
finance_polarities,finance_eval=calculate_new_lexicon_polarities(FINANCE_LEXICONS,POSITIVE_FINANCE,NEGATIVE_FINANCE,'label_propagate_prob')
standard_english_polarities,se_eval=calculate_new_lexicon_polarities(INQUIRER,POSITIVE_SE,NEGATIVE_SE,'label_propagate_prob')

## EVALUATING THE EFFECTIVENESS OF THE NEW LEXICON POLARITIES.

#### Calculates the ROC auc scores with the new polarities comparing to the earlier lexicon binary classification (1 = positive and 0 = negative).
#### Interpretation of the score. 
#### Higher the score, it means that new polarities (continous sentiment scores) confirms with the binary sentiment scores

In [24]:
### TASKS IN THE DOC : 

## 1. EXPLAIN ABOUT THE LEXICON INDUCING : LABEL PROPAGATION ALGORITHM
## 2. GIVES SOME UNDERLYING MATHEMATICS FROM THE RESEARCH PAPER REGARDING HOW SCORES ARE COMPUTED.

In [25]:
def evaluate_method_performance(polarities,INITIAL_LEXICON_LIB,domain,eval_words) : 
    
    ## EVALUATING THE EFFECTIVENESS OF THE NEW LEXICON POLARITIES.
    from evaluate_methods import binary_metrics,ternary_metrics

    acc, auc, avg_prec = binary_metrics(polarities, INITIAL_LEXICON_LIB, eval_words)
    if auc < 0.5:
        polarities = {word:-1*polarities[word] for word in polarities}
        acc, auc, avg_prec = binary_metrics(polarities, INITIAL_LEXICON_LIB, eval_words)

    print("============== DOMAIN : {} ==============".format(domain))
    print ("Binary metrics:")
    print( "==============")
    print ("Accuracy with optimal threshold: {:.4f}".format(acc))
    print ("ROC AUC Score: {:.4f}".format(auc))
    print ("Average Precision Score: {:.4f}".format(avg_prec))


    tau, cmn_f1, maj_f1, conf_mat = ternary_metrics(polarities, INITIAL_LEXICON_LIB, eval_words, tau_lexicon=THREE_WAY_LEXICON)
    print ("Ternary metrics:")
    print( "==============")
    print ("Majority macro F1 baseline {:.4f}".format(maj_f1))
    print ("Macro F1 with cmn threshold: {:.4f}".format(cmn_f1))
    if tau:
        print ("Kendall Tau {:.4f}".format(tau))
    print ("Confusion matrix: ")
    print (conf_mat)
    print( "Neg :", float(conf_mat[0,0]) / np.sum(conf_mat[0,:]))
    print ("Neut :", float(conf_mat[1,1]) / np.sum(conf_mat[1,:]))
    print ("Pos :", float(conf_mat[2,2]) / np.sum(conf_mat[2,:]))

In [26]:
evaluate_method_performance(finance_polarities,FINANCE_LEXICONS,'FINANCE',finance_eval)

============== DOMAIN : FINANCE ==============
Binary metrics:
Accuracy with optimal threshold: 1.7899
ROC AUC Score: 0.9576
Average Precision Score: 0.8367
Ternary metrics:
Majority macro F1 baseline 0.4642
Macro F1 with cmn threshold: 0.1180
Kendall Tau 0.3746
Confusion matrix: 
[[   0    1 2246]
 [   0    0    0]
 [   0    0  347]]
Neg : 0.0
Neut : nan
Pos : 1.0


In [28]:
evaluate_method_performance(standard_english_polarities,INQUIRER,' STANDARD ENGLISH ',se_eval)

============== DOMAIN :  STANDARD ENGLISH  ==============
Binary metrics:
Accuracy with optimal threshold: 1.1485
ROC AUC Score: 0.7891
Average Precision Score: 0.7536
Ternary metrics:
Majority macro F1 baseline 0.2497
Macro F1 with cmn threshold: 0.1024
Kendall Tau 0.3353
Confusion matrix: 
[[   0    1 1874]
 [   0    0 5106]
 [   0    0 1547]]
Neg : 0.0
Neut : 0.0
Pos : 1.0


/home/ubuntu/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## BASELINE : On the Movie Reviews using senti-wordnet lexicons

-  Accuracy: 0.6
-  Precision: 0.56
-  Recall: 0.93
-  F1 Score: 0.7

### This is to compute the effectiveness of binary sentiment scores provided a lexicon library.
### This can be used to see which lexicon libraries help achieving the closest sentiment scores.
### Thus a supervised algorithm and evaluation is the 

In [31]:
## INPUTS : 
## review = single sentence 
## lexicon_dict = dict of the lexicon with key as word and value as the polarity

def analyze_sentiment_domain(review,lexicon_dict,verbose=False):
    
    
    #review = normalize_accented_characters(review)
    #review = review.decode('utf-8')
    review = html_parser.unescape(review)
    review = strip_html(review)
    
    text_tokens = nltk.word_tokenize(review)
    tagged_text = nltk.pos_tag(text_tokens)
    pos_score = neg_score = token_count = obj_score = 0

    ## postitve polarity counts as positive and negative polarities counts as negative
    
    
    for token in text_tokens : 
        
        if token in lexicon_dict : 
            
            if lexicon_dict[token]>0 : 
                pos_score+=1
            elif lexicon_dict[token]<0:
                neg_score+=1

        token_count+=1
            
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
    if verbose:
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        
        
        
        sentiment_frame = pd.DataFrame([[final_sentiment,
                                         norm_pos_score, norm_neg_score,
                                         norm_final_score]],
                                         columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment',
                                                                       'Positive', 'Negative', 'Overall']], 
                                                              labels=[[0,0,0,0],[0,1,2,3]]))
        print (sentiment_frame)   
    return final_sentiment
            
                               

In [57]:
train_x,test_x,test_y=prepare_movie_dataset(0,1000,1000,2000)

sentiwordnet_predictions = [analyze_sentiment_domain(review,polarities) for review in test_x]
#evaluate_lexicons(test_y.tolist(),sentiwordnet_predictions,'positive','negative')

dataset size :  50000
Train_X :  1000
Test_X  :  1000


In [25]:
## TO DO : 

## 1. COMPUTE THESE SCORES AGAIN WITH THE NEW DOMAIN SPECIFIC (GLOVE BASED) EMBEDDINGS TO SEE SENTIMENTAL SCORE CHANGE.
## 2. GET THE TWITTER DATASET  AND DO AGAIN THE SAME THING. (TWITTER EMBEDDINGS/GLOVE EMBEDDINGS/SENTIWORDNET + TWITTER DATASET)


In [27]:
### FOR TWO DATASETS : 
### FOR THREE EMBEDDINGS : CUSTOM-MADE / GLOVE (1B) / ACTUAL-DOMAIN ONES.
### GET THESE SCORES (2*3 MATRIX OF SCORES)
### GET THE NEW POLARITIES AND USE THEM TO CALCULATE SENTIMENTAL SCORES(SENTENCE BASED).
### ALSO GET THE BINARY_METRICS FOR THE 

In [30]:
ss=swn.all_senti_synsets()
for s in ss : 
    print(s)

<able.a.01: PosScore=0.125 NegScore=0.0>
<unable.a.01: PosScore=0.0 NegScore=0.75>
<abaxial.a.01: PosScore=0.0 NegScore=0.0>
<adaxial.a.01: PosScore=0.0 NegScore=0.0>
<acroscopic.a.01: PosScore=0.0 NegScore=0.0>
<basiscopic.a.01: PosScore=0.0 NegScore=0.0>
<abducent.a.01: PosScore=0.0 NegScore=0.0>
<adducent.a.01: PosScore=0.0 NegScore=0.0>
<nascent.a.01: PosScore=0.0 NegScore=0.0>
<emergent.s.02: PosScore=0.0 NegScore=0.0>
<dissilient.s.01: PosScore=0.25 NegScore=0.0>
<parturient.s.02: PosScore=0.25 NegScore=0.0>
<dying.a.01: PosScore=0.0 NegScore=0.0>
<moribund.s.02: PosScore=0.0 NegScore=0.0>
<last.s.05: PosScore=0.0 NegScore=0.0>
<abridged.a.01: PosScore=0.0 NegScore=0.0>
<cut.s.03: PosScore=0.0 NegScore=0.0>
<half-length.s.02: PosScore=0.0 NegScore=0.0>
<potted.s.03: PosScore=0.0 NegScore=0.0>
<unabridged.a.01: PosScore=0.0 NegScore=0.0>
<full-length.s.02: PosScore=0.5 NegScore=0.0>
<absolute.a.01: PosScore=0.5 NegScore=0.0>
<direct.s.10: PosScore=0.75 NegScore=0.0>
<implicit.s.02

<eccrine.a.01: PosScore=0.25 NegScore=0.0>
<artesian.a.01: PosScore=0.0 NegScore=0.0>
<subartesian.a.01: PosScore=0.0 NegScore=0.125>
<live.a.02: PosScore=0.5 NegScore=0.0>
<in_play.s.01: PosScore=0.0 NegScore=0.0>
<living.s.06: PosScore=0.0 NegScore=0.125>
<dead.a.02: PosScore=0.0 NegScore=0.625>
<extinct.s.03: PosScore=0.0 NegScore=0.125>
<lifeless.s.04: PosScore=0.0 NegScore=0.75>
<out_of_play.s.01: PosScore=0.0 NegScore=0.0>
<alphabetic.a.02: PosScore=0.0 NegScore=0.0>
<abecedarian.s.01: PosScore=0.0 NegScore=0.0>
<alphabetized.s.01: PosScore=0.0 NegScore=0.0>
<analphabetic.a.02: PosScore=0.0 NegScore=0.125>
<altricial.a.01: PosScore=0.0 NegScore=0.0>
<precocial.a.01: PosScore=0.25 NegScore=0.0>
<altruistic.a.01: PosScore=0.875 NegScore=0.0>
<egoistic.a.01: PosScore=0.375 NegScore=0.0>
<self-absorbed.s.01: PosScore=0.5 NegScore=0.0>
<ambiguous.a.02: PosScore=0.125 NegScore=0.0>
<double-barreled.s.02: PosScore=0.5 NegScore=0.0>
<double-edged.s.01: PosScore=0.0 NegScore=0.0>
<enigmat

<pursuing.s.01: PosScore=0.0 NegScore=0.0>
<backed.a.01: PosScore=0.0 NegScore=0.0>
<hardbacked.s.01: PosScore=0.0 NegScore=0.125>
<high-backed.s.01: PosScore=0.125 NegScore=0.0>
<low-backed.s.01: PosScore=0.0 NegScore=0.125>
<razorback.s.01: PosScore=0.0 NegScore=0.0>
<spiny-backed.s.01: PosScore=0.0 NegScore=0.125>
<stiff-backed.s.01: PosScore=0.0 NegScore=0.0>
<straight-backed.s.01: PosScore=0.0 NegScore=0.0>
<backless.a.01: PosScore=0.0 NegScore=0.25>
<low-cut.s.02: PosScore=0.0 NegScore=0.0>
<backward.a.01: PosScore=0.0 NegScore=0.0>
<backswept.s.01: PosScore=0.0 NegScore=0.0>
<cacuminal.s.01: PosScore=0.0 NegScore=0.0>
<converse.s.02: PosScore=0.0 NegScore=0.0>
<inverse.s.01: PosScore=0.125 NegScore=0.0>
<rearward.s.02: PosScore=0.0 NegScore=0.0>
<receding.s.01: PosScore=0.0 NegScore=0.0>
<reflexive.s.02: PosScore=0.0 NegScore=0.0>
<regardant.s.01: PosScore=0.0 NegScore=0.0>
<retracted.s.01: PosScore=0.0 NegScore=0.0>
<retral.s.01: PosScore=0.0 NegScore=0.5>
<retroflex.s.01: PosS

<valiant.s.01: PosScore=0.625 NegScore=0.0>
<cowardly.a.01: PosScore=0.0 NegScore=0.5>
<caitiff.s.01: PosScore=0.0 NegScore=0.5>
<chicken.s.01: PosScore=0.0 NegScore=0.5>
<craven.s.01: PosScore=0.125 NegScore=0.375>
<dastard.s.01: PosScore=0.125 NegScore=0.5>
<faint.s.06: PosScore=0.125 NegScore=0.5>
<funky.s.04: PosScore=0.0 NegScore=0.25>
<poltroon.s.01: PosScore=0.625 NegScore=0.0>
<pusillanimous.s.01: PosScore=0.25 NegScore=0.625>
<gutsy.a.01: PosScore=0.125 NegScore=0.75>
<gutless.a.01: PosScore=0.375 NegScore=0.375>
<breast-fed.a.01: PosScore=0.0 NegScore=0.0>
<nursed.s.01: PosScore=0.0 NegScore=0.0>
<bottle-fed.a.01: PosScore=0.0 NegScore=0.0>
<breathing.a.01: PosScore=0.0 NegScore=0.0>
<sweet-breathed.s.01: PosScore=0.0 NegScore=0.125>
<breathless.a.01: PosScore=0.0 NegScore=0.75>
<asphyxiating.s.01: PosScore=0.125 NegScore=0.375>
<smothering.s.01: PosScore=0.125 NegScore=0.375>
<blown.s.02: PosScore=0.0 NegScore=0.0>
<crystalline.a.01: PosScore=0.0 NegScore=0.0>
<crystallized.

<certain.a.04: PosScore=0.0 NegScore=0.0>
<bound.s.04: PosScore=0.0 NegScore=0.0>
<doomed.s.04: PosScore=0.0 NegScore=0.0>
<foreordained.s.01: PosScore=0.0 NegScore=0.0>
<in_for.s.01: PosScore=0.0 NegScore=0.0>
<uncertain.a.04: PosScore=0.0 NegScore=0.5>
<chancy.s.02: PosScore=0.0 NegScore=0.0>
<contingent.s.03: PosScore=0.375 NegScore=0.5>
<up_in_the_air.s.02: PosScore=0.25 NegScore=0.0>
<certified.a.01: PosScore=0.625 NegScore=0.125>
<certifiable.s.02: PosScore=0.5 NegScore=0.0>
<certificated.s.01: PosScore=0.375 NegScore=0.125>
<credentialled.s.01: PosScore=0.375 NegScore=0.0>
<uncertified.a.01: PosScore=0.0 NegScore=0.25>
<inevitable.a.01: PosScore=0.0 NegScore=0.625>
<fatal.s.04: PosScore=0.125 NegScore=0.0>
<ineluctable.s.01: PosScore=0.0 NegScore=0.5>
<necessary.s.02: PosScore=0.125 NegScore=0.0>
<evitable.a.01: PosScore=0.625 NegScore=0.0>
<preventable.a.01: PosScore=0.625 NegScore=0.0>
<unpreventable.a.01: PosScore=0.125 NegScore=0.25>
<changeable.a.02: PosScore=0.375 NegScore

...

<all.a.01: PosScore=0.0 NegScore=0.0>
<each.s.01: PosScore=0.0 NegScore=0.0>
<every.s.01: PosScore=0.0 NegScore=0.0>
<every_last.s.01: PosScore=0.125 NegScore=0.0>
<every.s.02: PosScore=0.125 NegScore=0.0>
<sophisticated.a.01: PosScore=0.625 NegScore=0.0>
<blase.s.01: PosScore=0.25 NegScore=0.0>
<intelligent.s.02: PosScore=0.125 NegScore=0.0>
<polished.s.02: PosScore=0.5 NegScore=0.0>
<worldly-wise.s.01: PosScore=0.25 NegScore=0.0>
<naive.a.01: PosScore=0.625 NegScore=0.125>
<childlike.s.02: PosScore=0.875 NegScore=0.0>
<credulous.s.02: PosScore=0.5 NegScore=0.0>
<fleeceable.s.01: PosScore=0.5 NegScore=0.25>
<innocent.s.04: PosScore=0.375 NegScore=0.125>
<simple-minded.s.01: PosScore=0.25 NegScore=0.375>
<unsophisticated.s.01: PosScore=0.0 NegScore=0.625>
<sound.a.01: PosScore=0.75 NegScore=0.0>
<dependable.s.04: PosScore=0.5 NegScore=0.0>
<healthy.s.02: PosScore=0.5 NegScore=0.0>
<solid.s.11: PosScore=0.0 NegScore=0.0>
<stable.s.02: PosScore=0.5 NegScore=0.125>
<unsound.a.02: PosScore

<tense.a.01: PosScore=0.0 NegScore=0.0>
<aroused.s.03: PosScore=0.0 NegScore=0.0>
<cliff-hanging.s.01: PosScore=0.0 NegScore=0.5>
<taut.s.02: PosScore=0.0 NegScore=0.25>
<antsy.s.01: PosScore=0.0 NegScore=0.625>
<edgy.s.01: PosScore=0.0 NegScore=0.0>
<electric.s.02: PosScore=0.0 NegScore=0.0>
<isotonic.s.04: PosScore=0.0 NegScore=0.0>
<nervous.s.01: PosScore=0.125 NegScore=0.0>
<strained.s.02: PosScore=0.25 NegScore=0.0>
<unrelaxed.s.01: PosScore=0.0 NegScore=0.375>
<pumped-up.s.01: PosScore=0.0 NegScore=0.375>
<relaxed.a.01: PosScore=0.0 NegScore=0.625>
<degage.s.01: PosScore=0.625 NegScore=0.0>
<laid-back.s.01: PosScore=0.125 NegScore=0.5>
<unstrained.s.01: PosScore=0.25 NegScore=0.125>
<hypertonic.a.01: PosScore=0.0 NegScore=0.0>
<hypotonic.a.01: PosScore=0.0 NegScore=0.5>
<territorial.a.03: PosScore=0.0 NegScore=0.0>
<jurisdictional.s.01: PosScore=0.0 NegScore=0.0>
<regional.s.02: PosScore=0.0 NegScore=0.0>
<sectional.s.03: PosScore=0.0 NegScore=0.0>
<extraterritorial.a.01: PosScor

<anticlimactic.a.02: PosScore=0.125 NegScore=0.0>
<upmarket.a.01: PosScore=0.375 NegScore=0.0>
<upscale.s.01: PosScore=0.5 NegScore=0.0>
<downmarket.a.01: PosScore=0.0 NegScore=0.375>
<downscale.s.01: PosScore=0.0 NegScore=0.0>
<transitive.a.01: PosScore=0.0 NegScore=0.0>
<intransitive.a.01: PosScore=0.0 NegScore=0.5>
<translatable.a.01: PosScore=0.125 NegScore=0.0>
<untranslatable.a.01: PosScore=0.0 NegScore=0.5>
<ungulate.a.01: PosScore=0.125 NegScore=0.125>
<solid-hoofed.s.01: PosScore=0.125 NegScore=0.0>
<unguiculate.a.01: PosScore=0.0 NegScore=0.125>
<clawed.s.01: PosScore=0.0 NegScore=0.0>
<clawlike.s.01: PosScore=0.25 NegScore=0.25>
<up.a.01: PosScore=0.0 NegScore=0.0>
<ahead.s.01: PosScore=0.0 NegScore=0.0>
<aweigh.s.01: PosScore=0.0 NegScore=0.0>
<dormie.s.01: PosScore=0.0 NegScore=0.0>
<heavenward.s.01: PosScore=0.0 NegScore=0.0>
<risen.s.01: PosScore=0.0 NegScore=0.0>
<sprouted.s.01: PosScore=0.0 NegScore=0.0>
<upbound.s.01: PosScore=0.25 NegScore=0.0>
<upfield.s.01: PosScor

<rushy.s.01: PosScore=0.0 NegScore=0.375>
<scrabbly.s.01: PosScore=0.0 NegScore=0.125>
<sylvan.s.01: PosScore=0.0 NegScore=0.0>
<thicket-forming.s.01: PosScore=0.25 NegScore=0.5>
<timbered.s.02: PosScore=0.0 NegScore=0.25>
<woodsy.s.01: PosScore=0.0 NegScore=0.0>
<unwooded.a.01: PosScore=0.0 NegScore=0.25>
<unforested.s.01: PosScore=0.0 NegScore=0.625>
<untimbered.s.02: PosScore=0.0 NegScore=0.0>
<woody.a.01: PosScore=0.0 NegScore=0.0>
<ashen.s.02: PosScore=0.0 NegScore=0.0>
<beechen.s.01: PosScore=0.0 NegScore=0.0>
<birch.s.01: PosScore=0.0 NegScore=0.0>
<cedarn.s.01: PosScore=0.0 NegScore=0.0>
<ligneous.s.01: PosScore=0.0 NegScore=0.0>
<oaken.s.01: PosScore=0.0 NegScore=0.0>
<suffrutescent.s.01: PosScore=0.0 NegScore=0.0>
<wooden.s.01: PosScore=0.0 NegScore=0.0>
<nonwoody.a.01: PosScore=0.0 NegScore=0.625>
<herbaceous.s.01: PosScore=0.0 NegScore=0.125>
<pulpy.s.01: PosScore=0.0 NegScore=0.5>
<worldly.a.01: PosScore=0.0 NegScore=0.0>
<economic.s.04: PosScore=0.0 NegScore=0.0>
<materia

KeyboardInterrupt: 

### THE WHOLE USE OF THE METHODS IS THAT TO GET INSIGHT INTO THE CONTEXT-SENSITIVE INFORMATION.


### HOW? 

### 1. FIND THE LEXICONS IF BINARY 
### 2. WORD EMBEDDINGS TRAINED ON THE CONTEXT-MATERIAL.

### LABEL-PROPAGATE ALGORITHM TO MAKE LEXICONS FROM BINARY TO THE CONTINOUS SENTIMENT SCORES.

### USE SUM (P+V)/T OR NEURAL NETWORK TO OBTAIN THE SCORE FOR THE WHOLE SENTENCE SENTIMENT